## 다중 쿼리 RAG(Multi-Query RAG)

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

In [7]:
"""
Multi-Query RAG 에이전트 예제 (LangChain v1.0 + PDF)
- PDF 문서를 로드하여 Chroma에 저장
- MultiQueryRetriever를 사용해 다중 질의 수행
"""

import os
from datetime import datetime
from typing import List

# ① LangChain 핵심 및 OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# ② PDF 로더
from langchain_community.document_loaders import PyPDFLoader

# ③ MultiQueryRetriever (구버전 경로)
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

# ④ Agent 관련
from langchain.agents import create_agent
from langchain.tools import tool


# ---------------------------------------------------------
# 벡터스토어 준비
# ---------------------------------------------------------
def build_or_load_vectorstore(pdf_dir: str = "./docs",
                              persist_directory: str = "./chroma_mqrag_pdf") -> Chroma:
    """
    지정 폴더의 PDF 문서를 로드하여 Chroma 벡터스토어를 생성/로드한다.
    """
    os.makedirs(pdf_dir, exist_ok=True)

    # PDF 폴더 내 PDF가 없으면 안내
    pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]
    if not pdf_files:
        raise FileNotFoundError(
            f"{pdf_dir} 폴더에 PDF 문서가 없습니다. 예시용 PDF를 한두 개 넣어주세요."
        )

    # 모든 PDF 문서 로드
    docs_all = []
    for file in pdf_files:
        loader = PyPDFLoader(os.path.join(pdf_dir, file))
        docs = loader.load()
        docs_all.extend(docs)

    # 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=120,
        separators=["\n\n", "\n", " ", ""]
    )
    docs_split: List[Document] = splitter.split_documents(docs_all)

    # 임베딩 & 벡터스토어
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    if os.path.exists(persist_directory):
        vs = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
        if docs_split:
            vs.add_documents(docs_split)
    else:
        vs = Chroma.from_documents(
            documents=docs_split,
            embedding=embeddings,
            persist_directory=persist_directory
        )

    return vs


# ---------------------------------------------------------
# Multi-Query Retriever 준비
# ---------------------------------------------------------
def build_multi_query_retriever(vs: Chroma, llm: ChatOpenAI) -> MultiQueryRetriever:
    mq_prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
             "너는 정보 검색 전문가이다. 사용자의 질문을 다른 관점의 5개 검색 질의로 재작성한다. "
             "동의어, 상위/하위 개념, 원인/결과, 예시/반례, 비교/대조 관점 등을 섞어서 "
             "중복 없이 간결한 질의만 한 줄에 하나씩 출력한다."),
            ("human", "{question}")
        ]
    )

    base_retriever = vs.as_retriever(search_kwargs={"k": 5})
    mq_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm,
        prompt=mq_prompt
    )
    return mq_retriever


# ---------------------------------------------------------
# 답변 생성 프롬프트
# ---------------------------------------------------------
ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 근거 중심의 도우미이다. 주어진 문서 조각들을 바탕으로 응답을 생성한다. "
         "사실만 말하고 추측을 피한다. 답변 끝에 참고한 출처를 [출처 n] 형식으로 나열한다."),
        ("human",
         "질문:\n{question}\n\n"
         "다음은 참고 문서 조각이다:\n\n"
         "{context}\n\n"
         "요구사항:\n"
         "- 한국어로 간결하게 답한다.\n"
         "- 모르면 모른다고 말한다.\n"
         "- 답변 하단에 [출처 1], [출처 2] 형태로 인용한다.")
    ]
)


def format_context(docs: List[Document]) -> str:
    lines = []
    for idx, d in enumerate(docs, start=1):
        src = d.metadata.get("source", "unknown")
        lines.append(f"[출처 {idx}] source={src}\n{d.page_content[:500].strip()}\n")
    return "\n".join(lines)


# ---------------------------------------------------------
# 툴 정의: Multi-Query RAG 질의 도구
# ---------------------------------------------------------
@tool
def ask_multi_rag(question: str) -> str:
    """PDF 기반 Multi-Query RAG 질의"""
    global MQ_RETRIEVER, LLM
    retrieved = MQ_RETRIEVER.invoke(question)
    if not retrieved:
        return "관련 문서를 찾지 못했습니다."

    ctx = format_context(retrieved)
    prompt = ANSWER_PROMPT.format_messages(question=question, context=ctx)
    resp = LLM.invoke(prompt)
    return resp.content


# ---------------------------------------------------------
# 메인
# ---------------------------------------------------------
if __name__ == "__main__":
    LLM = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    vectorstore = build_or_load_vectorstore("./docs", "./chroma_mqrag_pdf")
    MQ_RETRIEVER = build_multi_query_retriever(vectorstore, LLM)

    agent = create_agent(LLM, [ask_multi_rag])

    query = "LangChain에서 Multi-Query RAG의 장점과 활용 방법은?"
    result = agent.invoke({
        "messages": [HumanMessage(content=query)]
    })

    print("\n=== 에이전트 답변 ===")
    print(result["messages"][-1].content)



=== 에이전트 답변 ===
LangChain에서 Multi-Query RAG(Reading and Generating) 시스템의 장점과 활용 방법에 대한 구체적인 정보는 제공되지 않았습니다. 그러나 일반적으로 Multi-Query RAG의 장점과 활용 방법에 대해 설명할 수 있습니다.

### 장점
1. **효율성**: 여러 쿼리를 동시에 처리할 수 있어, 정보 검색과 응답 생성의 속도를 높일 수 있습니다.
2. **정확성**: 다양한 쿼리를 통해 더 많은 정보를 수집하고, 이를 바탕으로 보다 정확한 응답을 생성할 수 있습니다.
3. **유연성**: 다양한 데이터 소스와 통합하여 사용할 수 있어, 다양한 도메인에서 활용할 수 있습니다.
4. **사용자 경험 향상**: 사용자에게 더 나은 응답을 제공함으로써, 전반적인 사용자 경험을 개선할 수 있습니다.

### 활용 방법
1. **정보 검색**: 사용자가 입력한 여러 질문에 대해 동시에 정보를 검색하고, 이를 종합하여 응답을 생성할 수 있습니다.
2. **대화형 시스템**: 챗봇이나 가상 비서와 같은 대화형 시스템에서 여러 쿼리를 처리하여 사용자와의 상호작용을 개선할 수 있습니다.
3. **데이터 분석**: 다양한 데이터 소스에서 정보를 수집하고 분석하여, 인사이트를 도출하는 데 활용할 수 있습니다.
4. **교육 및 학습**: 학습 자료를 제공하거나, 질문에 대한 답변을 제공하는 교육용 애플리케이션에서 사용할 수 있습니다.

이와 같은 장점과 활용 방법을 통해 LangChain의 Multi-Query RAG는 다양한 분야에서 유용하게 사용될 수 있습니다. 추가적인 정보가 필요하다면, 관련 문서를 참조하거나 구체적인 질문을 해주시면 더 도움을 드릴 수 있습니다.
